# 🧬 Stanford Ribonanza RNA Folding 2: Baseline Solution
이 노트북은 RNA 서열 데이터를 활용하여 3D 구조(좌표)를 예측하는 딥러닝 모델의 파이프라인을 다룹니다.

---

## 1. Import
필요한 라이브러리를 로드합니다.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## 2. 데이터 전처리 (Data Preprocessing)
서열 데이터를 정수 인코딩하고, 라벨(3D 좌표)과 매칭하여 학습용 데이터셋을 생성합니다.

In [ ]:
# 1. 데이터 로드
train_seq = pd.read_csv('train_sequences.csv')
train_labels = pd.read_csv('train_labels.csv')

print(f"Sequence data shape: {train_seq.shape}")
print(f"Label data shape: {train_labels.shape}")

# 2. 서열 토큰화 (A, C, G, U -> 1, 2, 3, 4)
mapping = {'A': 1, 'C': 2, 'G': 3, 'U': 4}
train_seq['tokenized'] = train_seq['sequence'].apply(lambda x: [mapping.get(base, 0) for base in x])

# 3. 라벨 전처리 (그룹화하여 좌표 리스트 생성)
print("Grouping labels by ID...")
coords_df = train_labels.groupby('ID')[['x_1', 'y_1', 'z_1']].apply(lambda x: x.values.tolist()).reset_index()
coords_df.columns = ['target_id', 'coordinates']

# 서열 데이터와 라벨 병합
train_df = pd.merge(train_seq, coords_df, on='target_id')

class RNADataset(Dataset):
    def __init__(self, sequences, coordinates, max_len=200):
        self.sequences = sequences
        self.coordinates = coordinates
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        coords = self.coordinates[idx]
        
        # Padding / Truncating Sequence
        if len(seq) < self.max_len:
            seq_padded = seq + [0] * (self.max_len - len(seq))
        else:
            seq_padded = seq[:self.max_len]
            
        # Padding / Truncating Coordinates
        coords = np.array(coords)
        if len(coords) < self.max_len:
            pad_width = ((0, self.max_len - len(coords)), (0, 0))
            coords_padded = np.pad(coords, pad_width, mode='constant', constant_values=0)
        else:
            coords_padded = coords[:self.max_len]
        
        return torch.tensor(seq_padded, dtype=torch.long), torch.tensor(coords_padded, dtype=torch.float32)

# 데이터 분할
train_idx, val_idx = train_test_split(range(len(train_df)), test_size=0.1, random_state=42)

train_dataset = RNADataset(train_df.iloc[train_idx]['tokenized'].values, train_df.iloc[train_idx]['coordinates'].values)
val_dataset = RNADataset(train_df.iloc[val_idx]['tokenized'].values, train_df.iloc[val_idx]['coordinates'].values)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## 3. 모델 선정 (Model Selection)
RNA의 2차 및 3차 구조는 염기 간의 원거리 상호작용(Long-range interaction)에 의해 결정되므로, **Transformer** 아키텍처가 매우 효과적입니다.

### 선정 모델: RNA Transformer Encoder
- **Self-Attention:** 서열 내 모든 위치 간의 관계를 학습합니다.
- **Output Head:** 각 토큰 위치에서 (x, y, z) 좌표를 예측하도록 구성합니다.

In [ ]:
class RNATransformer(nn.Module):
    def __init__(self, n_tokens=5, d_model=128, nhead=8, num_layers=4, dropout=0.1):
        super(RNATransformer, self).__init__()
        self.embedding = nn.Embedding(n_tokens, d_model)
        self.pos_encoder = nn.Parameter(torch.zeros(1, 1000, d_model))
        
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_model*4, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.fc_coords = nn.Linear(d_model, 3) # x, y, z 좌표 예측
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, d_model)
        x = x + self.pos_encoder[:, :x.size(1), :]
        
        x = self.transformer_encoder(x)
        # 각 위치별 좌표 예측 (batch_size, seq_len, 3)
        coords = self.fc_coords(x) 
        return coords

model = RNATransformer().to(device)
print(model)

## 4. 모델 학습 (Model Training)
MAE Loss를 최소화하도록 모델을 최적화합니다.

In [ ]:
criterion = nn.L1Loss() # MAE Loss
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

epochs = 5
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for seq, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        seq, target = seq.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(seq)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    print(f"Average Train Loss: {train_loss/len(train_loader):.4f}")

## 5. 모델 평가 (Model Evaluation)
Validation set을 활용하여 성능을 측정합니다.

In [ ]:
model.eval()
val_loss = 0
all_preds = []
all_targets = []

with torch.no_grad():
    for seq, target in val_loader:
        seq, target = seq.to(device), target.to(device)
        output = model(seq)
        loss = criterion(output, target)
        val_loss += loss.item()
        
        all_preds.append(output.cpu().numpy())
        all_targets.append(target.cpu().numpy())

print(f"Validation MAE: {val_loss/len(val_loader):.4f}")

# 시각화 (첫 번째 샘플의 x축 좌표)
plt.figure(figsize=(12, 6))
plt.plot(all_targets[0][0, :, 0], label='Actual X', color='blue', alpha=0.6)
plt.plot(all_preds[0][0, :, 0], label='Predicted X', color='red', linestyle='--')
plt.title('Actual vs Predicted X-coordinate (Sample 1)')
plt.xlabel('Residue Index')
plt.ylabel('X-coordinate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()